In [1]:
import json, re, os, sys
import torch


In [6]:
# minecraft
dirpath = os.getcwd().split('notebook')[0] + '/hierarchical_planning/'

def minecraft_data(verbose):
    f = open(dirpath + '/annotations/minecraft_text.annotated')
    lines = f.readlines()
    s = '\n'.join(line for line in lines)
    items = [item.split('\n') for item in s.split('No.')[1:]]
    items = [[i for i in item if len(i.strip()) > 0] for item in items]

    # dictionary of idxs -> text, connections, meta-info
    data = {}

    for idx in range(len(items)):
        text, connections, meta_info = [], [], []
        for item in items[idx]:
            if 'Text' in item:
                text.append(item.split('Text: ')[-1].strip())
            elif 'Connections' in item:
                temp = item.split('Connections: ')[-1].strip()
                connections.append(temp.split(' '))
            else:
                meta_info.append(item)
        data[idx] = {'text': text, 
                 'connections': connections, 
                 'meta':meta_info}
    
    if verbose:
        for idx in data:
            print(idx); print(data[idx]['text'])
            print(data[idx]['connections']); print('\n\n')
        
    return data


#minecraft_data(True)


In [2]:
import curses
import numpy as np
import pygame

def visualise(domain):
    if domain == 'minecraft':
        data = minecraft_data(False)
        
    WIDTH, HEIGHT = 10, 10
    WINDOW_WIDTH, WINDOW_HEIGHT = 5, 5
    N_WORKSHOPS = 3

    DOWN = 0
    UP = 1
    LEFT = 2
    RIGHT = 3
    USE = 4
    N_ACTIONS = USE + 1
    
    DIRT = 0
    GRASS = 1
    WATER = 2
    COAL = 3

    tilemap = [
            [GRASS, COAL, DIRT],
            [WATER, WATER, GRASS],
            [COAL, GRASS, WATER],
            [DIRT, GRASS, COAL],
            [GRASS, WATER, DIRT]
          ]
    BLACK = (0, 0, 0)
    BROWN = (153, 76, 0)
    GREEN = (0, 255, 0)
    BLUE = (0, 0, 255)
    
    colours = {
            DIRT:BROWN,
            GRASS:GREEN,
            WATER:BLUE,
            COAL:BLACK
          }
    
    TILESIZE = 40
    MAPWIDTH = 3
    MAPHEIGHT = 5

    #set up the display
    pygame.init()
    DISPLAYSURF = pygame.display.set_mode((MAPWIDTH*TILESIZE,MAPHEIGHT*TILESIZE))

    while True:
        #get all the user events
        for event in pygame.event.get():
            #if the user wants to quit
            if event.type == pygame.QUIT:
                #and the game and close the window
                pygame.quit()
                sys.exit()
                
        #loop through each row
        for row in range(MAPHEIGHT):
            #loop through each column in the row
            for column in range(MAPWIDTH):
                #draw the resource at that position in the tilemap, using the correct colour
                pygame.draw.rect(DISPLAYSURF, colours[tilemap[row][column]], (column*TILESIZE,row*TILESIZE,TILESIZE,TILESIZE))

        #update the display
        pygame.display.update()

    
#visualise('minecraft')

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# monty

def monty_data():
    f = open(dirpath + '/annotations/mr_text.annotated')
    lines = f.readlines()
    s = '\n'.join(line for line in lines)
    tag_finder = re.compile('(?<=<p>).*(?=</p>)')
    words = []

    '''
    for line in lines:
        m = tag_finder.search(line)
        if m != None:
            item = m.group(0).strip()
            words.append(item)
    '''

    items = [item.split('<p')[-1] for item in s.split('/p>')]
    return items
